## SEIR Model

Based on http://gabgoh.github.io/COVID/index.html, with modifications

In [ ]:
%load_ext lab_black
import numpy as np
import pandas as pd
from pandas.testing import assert_series_equal
from seir import simulator

In [ ]:
# Calibration: Alternating regimes of exposure and social distancing (see r_0_values)

# Note:
# - Hospitalisation rate is assumed to be p_severe + p_fatal = 20%
# - We might need to modify the model further to differentiate between hospitalised
# cases (here 20%) and the much smaller subset of ICU cases. A quick hack could be
# to just multiply 'I_severe_hospital' by a conditional ICU fraction we find in the
# literature, to then add all of 'I_fatal_hospital', and to call this time series ICUs

periods = 100
dT = 0.01

parameters = {
    "r_0_days": [21, 42, 63, 100],  # note: the last entry has to equal 'periods'
    "r_0_values": [3.5, 1.25, 2.5, 1.25],
    "t_infectious": 2.9,
    "t_incubation": 5.2,
    "p_severe": 0.18,
    "p_fatal": 0.02,
    "t_recovery_mild": 15 - 2.9,
    "t_recovery_severe": 31.5 - 2.9,
    "t_hospital_lag": 5,
    "t_death": 32 - 2.9,
}

N = 10000
start = {
    "T": 0.0,
    "S": 1.0 - 1 / N,
    "E": 1.0 / N,
    "I": 0.0,
    "I_mild": 0.0,
    "I_severe_home": 0.0,
    "I_severe_hospital": 0.0,
    "I_fatal_home": 0.0,
    "I_fatal_hospital": 0.0,
    "R_from_mild": 0.0,
    "R_from_severe": 0.0,
    "Dead": 0,
}

In [ ]:
# Simulation
simulations = simulator(parameters, start, periods, dT)

In [ ]:
# Additional variables
simulations = (
    pd.DataFrame.from_dict(simulations)
    .set_index("T")
    .assign(
        Hospitalised=lambda x: x["I_severe_hospital"] + x["I_fatal_hospital"],
        total_start=lambda x: x.iloc[0]["S"] + x.iloc[0]["E"],
        total=lambda x: x["S"]
        + x["E"]
        + x["I_mild"]
        + x["I_severe_home"]
        + x["I_severe_hospital"]
        + x["I_fatal_home"]
        + x["I_fatal_hospital"]
        + x["R_from_mild"]
        + x["R_from_severe"]
        + x["Dead"],
    )
)

In [ ]:
# Check for constant population
assert_series_equal(simulations["total"], simulations["total_start"])

In [ ]:
simulations[["S", "I", "E"]].plot()

In [ ]:
simulations[["I", "Hospitalised"]].plot()

In [ ]:
simulations[["R_from_mild", "R_from_severe", "Dead"]].plot()

In [ ]:
# Code snippet of the version from the blog/dashboard (issue: fatalities seem not to
# show up in hospitals)


# Flows into three different subclasses of infectious
# A: Mild course
dI_mild = (
    p_mild * gamma * I - (1 / t_recovery_mild) * I_mild
) * dT

# B: Severe  course (has two steps)
dI_severe_home = (
    p_severe * gamma * I - (1 / t_hospital_lag) * I_severe_home
) * dT
dI_severe_hospital = (
    (1 / t_hospital_lag) * I_severe_home
    - (1 / t_recovery_severe) * I_severe_hospital
) * dT

# C: Fatal course
dI_fatal = (p_fatal * gamma * I - (1 / t_death) * I_fatal) * dT

# Flows into recovery or death
dR_from_mild = ((1 / t_recovery_mild) * I_mild) * dT
dR_from_severe = (
    (1 / t_recovery_severe) * I_severe_hospital
) * dT
dDead = ((1 / t_death) * I_fatal) * dT